## Code for CO2 Bubbling -> ECMS Demonstration

Go to *Platformio > Devices* to find COM port address.

In [6]:
%load_ext autoreload
%autoreload 2

from src.pump_controller import ECMSController

ecms_handler = ECMSController(COM="/dev/cu.wchusbserial110", sim=False)

INFO:root:Configuring EC-MS controller serial port..
INFO:root:Attempting to open EC-MS controller serial port..


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:root:Successful response from EC-MS controller: # SDL Board Ready


## Set CO2 Bubbling & Dilution Parameters

In [2]:
# Dilution parameters
total_volume = 10 #ml
khco3_volume = 0.2 * total_volume #ml
water_volume = total_volume - khco3_volume #ml

# CO2 parameters
CO2_bubbling = 30 #s

mixture_to_ecms = 550 #mm (slightly faster)

### Turn on CO2 Valve

In [ ]:
ecms_handler.releaseCO2(duration=5)

## Prime Water & Chemical Lines

In [ ]:
# Deprime first for safety
ecms_handler.deprime()

# Primes up to mixing chamber
ecms_handler.prime()

## Run Procedure

In [3]:
# Create mixture
ecms_handler.addChemical(fluid_vol=khco3_volume)
ecms_handler.addWater(fluid_vol=water_volume)

# Bubble CO2
ecms_handler.releaseCO2(duration=CO2_bubbling)

# Transfer for measuring
ecms_handler.sendToWaste()
ecms_handler.sendToPH()

# Take measurement
#while not ecms_handler.checkPH():
    #ecms_handler.releaseCO2(duration=CO2_bubbling/3)
    #ecms_handler.sendToWaste()
    #ecms_handler.sendToPH()

# Remove electrolyte and wet pH probe
ecms_handler.sendToWaste()
ecms_handler.refreshWater()

# Transfer to ECMS (prime mixture up to cell)
ecms_handler.transferToECMS(tube_length=mixture_to_ecms, flow_rate=0.1)


INFO:root:Pumping 2.0ml of chemical to mixing pot..
INFO:root:Successful response from EC-MS controller: # Chemical added
INFO:root:Pumping 8.0ml of water to mixing pot..
INFO:root:Successful response from EC-MS controller: # Water added
INFO:root:Opening CO2 valve for 30s..
INFO:root:Successful response from EC-MS controller: # CO2 dosing complete
INFO:root:Sending to waste from pH chamber (10s)..
INFO:root:Successful response from EC-MS controller: # Sent to Waste
INFO:root:Pumping 5ml of chemical to pH chamber..
INFO:root:Successful response from EC-MS controller: # Mixture sent to pH chamber
INFO:root:Sending to waste from pH chamber (10s)..
INFO:root:Successful response from EC-MS controller: # Sent to Waste
INFO:root:Sending to water to pH chamber (4s)..
INFO:root:Successful response from EC-MS controller: # Refreshed Water
INFO:root:Pumping 0ml of saturated mixture to EC-MS..
INFO:root:Successful response from EC-MS controller: # Mixture sent to ECMS


## Repeated Cell Flushing & Measurements

In [ ]:
cell_volume = 0.1 #ml
flush_volume = 0.2 #ml
repeats = 5

### Fill Cell

In [ ]:
ecms_handler.transferToECMS(fluid_vol=cell_volume)

INFO:root:Pumping 0ml of saturated mixture to EC-MS..
INFO:root:Successful response from EC-MS controller: # Mixture sent to ECMS


### Flush Cell

In [ ]:
ecms_handler.transferToECMS(fluid_vol=flush_volume)

### Return remainder to mixing chamber

In [ ]:
ecms_handler.transferToECMS(tube_length=-mixture_to_ecms, overpump=1.5)

### Deprime

In [ ]:
# Deprime
ecms_handler.deprime()

# Empty to waste & refresh water in pH tube
ecms_handler.sendToPH()
ecms_handler.sendToWaste()
ecms_handler.refreshWater()